In [2]:
import sys

sys.path.append('/Users/floyd/Desktop/DeepIV/deepiv')

In [4]:
from __future__ import print_function

from models import Treatment, Response
import densities as densities

from keras.layers import Input, Dense
from keras.models import Model

import data_generator

Using TensorFlow backend.


In [5]:
epochs = 100
# Very simple linear data where two stage least squares is optimal
x, z, t, y, g_true = data_generator.linear_data(n=1000, seed=1)

print("Starting experiment with linear data\n" + "-"*50 +
"\nData shapes:\n\
Features:{x},\n\
Instruments:{z},\n\
Treament:{t},\n\
Response:{y}".format(**{'x':x.shape, 'z':z.shape,
                        't':t.shape, 'y':y.shape}))

Starting experiment with linear data
--------------------------------------------------
Data shapes:
Features:(1000, 0),
Instruments:(1000, 1),
Treament:(1000, 1),
Response:(1000, 1)


In [6]:
# Build and fit treatment model
instruments = Input(shape=(z.shape[1],))
x = Dense(64, activation='relu')(instruments)
x = Dense(64, activation='relu')(x)
#est_treat = Dense(1)(x)
est_treat = densities.mixture_of_gaussian_output(x, 10)

treatment_model = Treatment(inputs=[instruments], outputs=est_treat)
treatment_model.compile('adam',
                        loss="mixture_of_gaussians",
                        n_components=10)
treatment_model.fit([z],t, epochs=epochs)


Epoch 1/100
1000/1000 [==============================] - 1s 603us/step - loss: 1.4738
Epoch 2/100
1000/1000 [==============================] - 0s 146us/step - loss: 1.4676
Epoch 3/100
1000/1000 [==============================] - 0s 60us/step - loss: 1.4652
Epoch 4/100
1000/1000 [==============================] - 0s 61us/step - loss: 1.4632
Epoch 5/100
1000/1000 [==============================] - 0s 59us/step - loss: 1.4623
Epoch 6/100
1000/1000 [==============================] - 0s 48us/step - loss: 1.4611
Epoch 7/100
1000/1000 [==============================] - 0s 53us/step - loss: 1.4595
Epoch 8/100
1000/1000 [==============================] - 0s 68us/step - loss: 1.4581
Epoch 9/100
1000/1000 [==============================] - 0s 62us/step - loss: 1.4582
Epoch 10/100
1000/1000 [==============================] - 0s 54us/step - loss: 1.4589
Epoch 11/100
1000/1000 [==============================] - 0s 65us/step - loss: 1.4569
Epoch 12/100
1000/1000 [==============================] - 0s 

1000/1000 [==============================] - 0s 50us/step - loss: 1.4553
Epoch 97/100
1000/1000 [==============================] - 0s 49us/step - loss: 1.4554
Epoch 98/100
1000/1000 [==============================] - 0s 60us/step - loss: 1.4552
Epoch 99/100
1000/1000 [==============================] - 0s 60us/step - loss: 1.4562
Epoch 100/100
1000/1000 [==============================] - 0s 58us/step - loss: 1.4559


In [7]:
# Build and fit response model
x = Dense(64, activation='relu')(instruments)
x = Dense(64, activation='relu')(x)
est_resp = Dense(1)(x)
response_model = Response(treatment=treatment_model,
                          inputs=[instruments],
                          outputs=est_resp)
response_model.compile('adam', loss='mse')
response_model.fit([z], y, epochs=epochs, verbose=1,
                    batch_size=100, samples_per_batch=2)

Epoch 1/100
10/10 [==============================] - 0s 25ms/step - loss: 0.6825
Epoch 2/100
10/10 [==============================] - 0s 10ms/step - loss: 0.6904
Epoch 3/100
10/10 [==============================] - 0s 11ms/step - loss: 0.7080
Epoch 4/100
10/10 [==============================] - 0s 10ms/step - loss: 0.6843
Epoch 5/100
10/10 [==============================] - 0s 10ms/step - loss: 0.6346
Epoch 6/100
10/10 [==============================] - 0s 10ms/step - loss: 0.6717
Epoch 7/100
10/10 [==============================] - 0s 10ms/step - loss: 0.6592
Epoch 8/100
10/10 [==============================] - 0s 10ms/step - loss: 0.7008
Epoch 9/100
10/10 [==============================] - 0s 11ms/step - loss: 0.6778
Epoch 10/100
10/10 [==============================] - 0s 10ms/step - loss: 0.6632
Epoch 11/100
10/10 [==============================] - 0s 11ms/step - loss: 0.6725
Epoch 12/100
10/10 [==============================] - 0s 10ms/step - loss: 0.6800
Epoch 13/100
10/10 [=====

In [8]:
def datafunction(n, s, images=False, test=False):
    return data_generator.linear_data(n=n, seed=s)

oos_perf = data_generator.monte_carlo_error(lambda x,z,t: response_model.predict([t]), datafunction, has_latent=False, debug=False)
print("Out of sample performance evaluated against the true function: %f" % oos_perf)


Out of sample performance evaluated against the true function: 22.691473


In [11]:
response_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                128       
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 4,353
Trainable params: 4,353
Non-trainable params: 0
_________________________________________________________________
